# Forensic-Playbook Labs — 00 · Introduction & Setup

**Generated:** 2025-10-10 12:46:19Z

Welcome! These notebooks are designed to be **deterministic** and **guarded**:
- Dry-Run first, then (optionally) execute for real.
- No binary fixtures; synthetic or JSON fallbacks only.
- Clear provenance of any artifacts generated.

## What you'll need
```
python -m venv .venv
source .venv/bin/activate
pip install -e .  # from the Forensic-Playbook repo root
```
If the `forensic-cli` is on your PATH, the following checks should pass.

In [ ]:
# Verify that we can import the SDK and invoke CLI help (tolerant).
import shutil, subprocess, json, sys

print("Python:", sys.version)
for tool in ["forensic-cli"]:
    path = shutil.which(tool)
    print(f"{tool}: {path or 'NOT FOUND (ok for SDK-only cells)'}")

try:
    out = subprocess.run(["forensic-cli", "--help"], capture_output=True, text=True, check=False)
    print(out.stdout[:800])
except FileNotFoundError:
    print("forensic-cli not found — SDK-only cells will still run.")

## Configuration precedence (CLI > YAML > Defaults)
We demonstrate reading the active configuration using the SDK.

In [ ]:
# SDK peek at configuration (no writes).
from pathlib import Path
try:
    from forensic.core.config import load_config
    cfg = load_config(Path.cwd() / "config")
    print(json.dumps(cfg.get('framework', {}), indent=2, sort_keys=True)[:1200])
except Exception as e:
    print("Config not available yet:", e)